In [9]:
import sys
sys.path.append('../modules')
%matplotlib inline
import pandas as pd

In [10]:
from modules.Reports.CoinbaseReport import CoinbaseReport
from modules.Reports.ReportGenerator import ReportGenerator
from modules.Pipelines.NetworkDataPipe import NetworkDataPipe
from modules.Visualizers.LineChart import LineChart
from modules.Visualizers.SubPlot import SubPlot
from modules.Visualizers.Table import Table
from modules.Transformers.Divider import Divider
from modules.Transformers.Multiplier import Multiplier

ModuleNotFoundError: No module named 'modules'

In [ ]:
api_key = 'KKzV6V2DTY87v3m1dGZu'
start = '2019-01-01'
asset_sample = [
    'btc',
    'xrp',
    'eth',
    'bch',
    'ltc',
#     'eos',
    'xlm',
    'xtz',
    'bsv',
    'ada',
    'zec',
    'link',
    'dcr',
    'mkr',
    'bat',
    'usdt',
    'usdt_eth',
    'dai',
    'usdc'
]

valuation = [
    'CapRealUSD',
    'CapMrktCurUSD',
    'CapMVRVCur',
#     'NVTAdj',
    'VtyDayRet30d',
    'PriceUSD'
    
    
]

economics = [
    'TxCnt',
    'TxTfrCnt',
    'TxTfrValAdjUSD',
    'IssContPctAnn',
    'FeeMedUSD',
#     'VelActAdj1yr',
#     'VelCurAdj1yr',
#     'TxTfrValAdjByte'
]

usage = [
    'AdrActCnt',
    'AdrBalUSD10Cnt',
    'AdrBalUSD1MCnt',
    'BlkSizeByte'
]

# 'FeesToBlockReward', 'DeltaCap'
security_and_health = [
        'HashRate',
        'DiffMean',
        'FeeTotUSD',
        'FeeRevPct',
        'RevUSD',
#         'RevAllTimeUSD'
]

extra= [
    'SplyCur',
    'SplyAct30d',
]

metrics = valuation + economics + usage + security_and_health + extra

assets = asset_sample
growth = True
# market_share = False
# aggregate = False
# aggregated_assets_name = None
# seven_day_rolling_average = True
# log_y_axis = False
# y2_axis_columns = None

In [ ]:
df = NetworkDataPipe(api_key=api_key, metrics=metrics, assets=assets, start=start).run()
for asset in assets:
    df = Divider(column_a=f'{asset}.SplyAct30d', column_b=f'{asset}.SplyCur', new_column_name=f'{asset}.SplyAct30dPercent').transform(df)
    df = Multiplier(column=f'{asset}.SplyAct30dPercent', multiplier=100).transform(df)
    
df.head()

In [ ]:
table_data= df.loc['2019-12-18':'2019-12-18']
# last_val = df['btc.PriceUSD'].iloc[df['btc.PriceUSD'].size - 1]
# last_val
table_data.head()

In [ ]:
def map_metric_name(metric):
    metric_mappings = {
        'Asset': 'Asset',
        'CapRealUSD' : 'Realized Cap',
        'CapMrktCurUSD': 'Market Cap',
        'CapMVRVCur': 'MVRV',
        'VtyDayRet30d': 'Volatility, 30d',
        'PriceUSD': 'Price, USD',
        'AdrActCnt': 'Active Addresses',
        'AdrBalUSD10Cnt': 'Addresses w/ Balance >= $10',
        'AdrBalUSD1MCnt': 'Addresses w/ Balance >= $1M',
        'SplyAct30d': 'Active Supply, 30d',
        'SplyAct30dPercent' : 'Active Supply Percentage, 30d',
        'BlkSizeByte': 'Block Size, Bytes',
        'TxCnt': 'Transaction Count',
        'TxTfrCnt': 'Transfer Count',
        'TxTfrValAdjUSD': 'Adjusted Transfer Value',
        'IssContPctAnn': 'Issuance, Annualized',
        'FeeMedUSD': 'Median Transaction Fee',
        'HashRate': 'Hash Rate',
        'DiffMean': 'Difficulty, Mean',
        'FeeTotUSD': 'Fees, USD',
        'FeeRevPct': 'Fee to Revenue Percentage',
        'RevUSD': 'Miner Revenue, USD'
        
    }

    return metric_mappings[metric] if metric in list(metric_mappings.keys()) else metric

In [ ]:
def create_year_end_table(df, assets, metrics):
    table_df = pd.DataFrame(columns=['Asset'])

    for asset in assets:
        _new_row = {}
        _new_row['Asset'] = asset
        for metric in metrics:
            _new_row[metric] = df.iloc[0][f'{asset}.{metric}']
        table_df = table_df.append(_new_row, ignore_index=True)
        table_df.sort_values(by=list(table_df.columns)[1], ascending=False, inplace=True)
        
    mapper =  {
       'AdrActCnt': '{0:,.0f}',
       'AdrBalUSD10Cnt': '{0:,.0f}',
       'AdrBalUSD1MCnt': '{0:,.0f}',
#        'SplyAct30d': '{0:,.2f}',
       'BlkSizeByte': '{0:,.0f}',
       'SplyAct30dPercent': '{0:.2f}%',
    }

    for key, value in mapper.items():
        table_df[key] = table_df[key].apply(value.format)
    
    table_df.columns = map(map_metric_name, list(table_df.columns))
    
    _table = Table(df=table_df, title='Tab;e', table_headers=list(table_df.columns), table_columns=list(table_df.columns)).run()
    _table.fig.update_layout(height=600)
    _table.export('Test-Table')
    return _table.show()

create_year_end_table(df=table_data, assets=assets, metrics=usage + ['SplyAct30dPercent'])

In [ ]:
# def create_table(self):
#     fig = go.Figure(data=[go.Table(
#         header={
#             'values': self.table_headers,
#             'fill_color': 'skyblue',
#             'align': 'left'
#         },
#         cells={
#             'values': [self.df[column] for column in self.table_columns],
#             'fill_color': 'snow',
#             'align': 'left'
#         }
#     )])

In [ ]:
growth = True
seven_day_rolling_average = True
y2_price_trace = False

subplot_visuals = []

for metric in metrics + ['SplyAct30dPercent']:
    metric_title = map_metric_name(metric)
    _subplot = SubPlot(df=df, title=f'{metric_title}, 2019', number_of_rows=6, number_of_columns=3, columns_to_plot=[f'{asset}.{metric}' for asset in assets], growth=growth, seven_day_rolling_average=seven_day_rolling_average, y2_price_trace=y2_price_trace)
    subplot_visuals.append(_subplot)
    
for visual in subplot_visuals:  
    _visual = visual.run()
    _visual.fig.update_layout(
        showlegend=False,
        height= 900,
        width = 800,
        legend = {
#         'orientation': 'v',
#         'xanchor': 'left',
        'xanchor': 'center',
        'yanchor': 'bottom',
        'x': .5,
        'y': -.25,
        'orientation': 'h'
        })
    _visual.show()

In [ ]:
# for metric in metrics:
#     LineChart(df=df, title=f'{metric}, 2019', y_columns=[f'{asset}.{metric}' for asset in assets], growth=growth).run().show()

In [ ]:
active_supply_metrics = ['SplyAct30d', 'SplyAct180d', 'SplyAct1yr', 'SplyAct2yr', 'SplyAct3yr', 'SplyAct4yr', 'SplyAct5yr']
active_supply_df = NetworkDataPipe(api_key=api_key, metrics=active_supply_metrics, assets=['btc', 'eth'], start=start).run()

LineChart(df=active_supply_df, title=f'BTC Active Supply, 2019', y_columns=[f'btc.{metric}' for metric in active_supply_metrics], growth=False, filled_area=True).run().show()

In [ ]:

addresses_with_balance_greater_than_X_ntv_metrics = ['AdrBalNtv0.1Cnt', 'AdrBalNtv1Cnt', 'AdrBalNtv10Cnt', 'AdrBalNtv100Cnt', 'AdrBalNtv1KCnt']

addresses_with_balance_greater_than_X_usd_metrics = ['AdrBalUSD1Cnt', 'AdrBalUSD10Cnt', 'AdrBalUSD100Cnt', 'AdrBalUSD1KCnt', 'AdrBalUSD10KCnt', 'AdrBalUSD100KCnt', 'AdrBalUSD1MCnt', 'AdrBalUSD10MCnt']

addresses_with_balance_greater_than_1_in_X_ntv_metrics = ['AdrBal1in100KCnt', 'AdrBal1in1MCnt', 'AdrBal1in10MCnt', 'AdrBal1in100MCnt', 'AdrBal1in1BCnt']

dist_metrics = addresses_with_balance_greater_than_X_ntv_metrics + addresses_with_balance_greater_than_X_usd_metrics + addresses_with_balance_greater_than_1_in_X_ntv_metrics

distribution_df = NetworkDataPipe(api_key=api_key, metrics=dist_metrics, assets=['btc', 'eth'], start=start).run()

LineChart(df=distribution_df, title=f'BTC Addresses w/ Balance > X, NTV Units, 2019', y_columns=[f'btc.{metric}' for metric in addresses_with_balance_greater_than_X_ntv_metrics], growth=False, filled_area=True).run().show()